This is my Implementation of Naive Bayes Classifier using Bags of word appoach.

I'm Using Glob library to get all the filenames from a directory, other than that i'm using Urduhack for data pre processing.

In [ ]:

import glob #Use to get all filenames from a directory
!pip install urduhack[tf]
import urduhack
urduhack.download()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In this cell my implementation of Niave Bayes is written. It is quite a large code as i tried multiple approaches and they required lots of helper function. 

My approaches Include:

On Multinomial Naive Bayes:

Training and Testing without removing stopwords

Training (without stopwords) and testing (with stopwords)

Training (without stopwords) and testing (without stopwords)

On Boolean Naive Bayes:

Removing Repitition from the data


Training and Testing without removing stopwords

Training (without stopwords) and testing (with stopwords)

Training (without stopwords) and testing (without stopwords)

On Boolean Naive Bayes and ignoring negations as a bonus task:

Training and Testing without removing stopwords

Training (without stopwords) and testing (with stopwords)

Training (without stopwords) and testing (without stopwords)

*Functionality and working of each function is provided in its defination


In [ ]:
class NaiveBayes:
  def __init__(self,DirFake,DirReal,DirFakeTest,DirRealTest,StopWordsDir):
    '''
    This is the constuctor, i loaded all the textfile names from the directories
    and loaded all the stopword we were provided
    '''
    self.fakeDS=glob.glob(DirFake+"/*")
    self.realDS=glob.glob(DirReal+"/*")
    self.fakeDSTest=glob.glob(DirFakeTest+"/*")
    self.realDSTest=glob.glob(DirRealTest+"/*")
    self.stopWords=[]
    with open(StopWordsDir, 'rt', encoding="utf-8") as f:
        self.stopWords=f.readlines()
    self.stopWords=[x.strip() for x in self.stopWords]
  def removePunctuation(self,text):# Removes Punctuation
    from urduhack.preprocessing import remove_punctuation
    return remove_punctuation(text)
  def removeNumbers(self,text):# Removes Numbers
    from urduhack.preprocessing import replace_phone_numbers
    return replace_phone_numbers(text)
  def removeEnglish(self,text):
    from urduhack.preprocessing import remove_english_alphabets
    return urduhack.preprocessing.remove_english_alphabets(text)
  def wordTokenize(self,text):
    from urduhack.tokenization import sentence_tokenizer
    sentences = sentence_tokenizer(text)
    from urduhack.tokenization import word_tokenizer
    lX=[]
    for sent in sentences:
      lX.append(sent.split(' '))
    flat_list = [item for sublist in lX for item in sublist]
    return flat_list
  def normalizeText(self,text):# Control Function
    from urduhack.normalization import normalize_characters
    x = normalize_characters(text)
    x=self.removePunctuation(text)
    x=self.removeNumbers(x)
    x=self.removeEnglish(x)
    x=self.wordTokenize(x)
    return x
  def loadTrainData(self):
    '''
    Loading Fake and Real Train data into lists of sublists where
    each sublist contains the normalized and tokenized content of one file
    '''
    self.urduListFakeTrain=[]
    for textFile in self.fakeDS:
      urdu_text = open(textFile, 'r', encoding="utf8").read()
      self.urduListFakeTrain.append(self.normalizeText(urdu_text))
    self.urduListRealTrain=[]
    for textFile in self.realDS:
      urdu_text = open(textFile, 'r', encoding="utf8").read()
      self.urduListRealTrain.append(self.normalizeText(urdu_text))
  def train(self):
    '''
    First i created a FakeVocabularly and a Real Vocabularly. Then from those
    i created a global vocab named myVocab after that i created 2 bag of words which contained
    words from myVocab, one was for fakeData and one for realData.
    After that i removed stopwords from myVocab and recreated 2 more bag of words in 
    the same way i did before.
    '''
    self.fakeVocab=[]
    for line in self.urduListFakeTrain:
      for word in line:
        self.fakeVocab.append(word)
    self.realVocab=[]
    for line in self.urduListRealTrain:
      for word in line:
        self.realVocab.append(word)
    self.myVocab=self.fakeVocab+self.realVocab
    self.fakeBagofWords={}
    for word in self.myVocab:
      if word in self.fakeVocab:
        if word not in self.fakeBagofWords:
          self.fakeBagofWords[word]=1
        else:
          self.fakeBagofWords[word]+=1
      else:
        self.fakeBagofWords[word]=1
    self.realBagofWords={}
    for word in self.myVocab:
      if word in self.realVocab:
        if word not in self.realBagofWords:
          self.realBagofWords[word]=1
        else:
          self.realBagofWords[word]+=1
      else:
        self.realBagofWords[word]=1
    ##removing Stop Words from Vocab
    self.myVocabStop=[]
    for word in self.myVocab:
      if word not in self.stopWords:
        self.myVocabStop.append(word)
    self.fakeBagofWordStop={}
    for word in self.myVocabStop:
      if word in self.fakeVocab:
        if word not in self.fakeBagofWordStop:
          self.fakeBagofWordStop[word]=1
        else:
          self.fakeBagofWordStop[word]+=1
      else:
        self.fakeBagofWordStop[word]=1
    self.realBagofWordStop={}
    for word in self.myVocabStop:
      if word in self.realVocab:
        if word not in self.realBagofWordStop:
          self.realBagofWordStop[word]=1
        else:
          self.realBagofWordStop[word]+=1
      else:
        self.realBagofWordStop[word]=1
  def checkVar(self): #Just a function to check data's length
    print(len(self.myVocab))
    print(len(self.myVocabStop))
    print(len(self.fakeBagofWords))
    print(len(self.realBagofWords))
    print(len(self.fakeBagofWordStop))
    print(len(self.realBagofWordStop))
  def loadtestData(self):
    '''
    I'm loading test data the same way i loaded trained data into to seperate
    list of sublist names of which can be easily recognised
    '''
    self.urduListFakeTest=[]
    for textFile in self.fakeDSTest:
      urdu_text = open(textFile, 'r', encoding="utf8").read()
      self.urduListFakeTest.append(self.normalizeText(urdu_text))
    self.urduListRealTest=[]
    for textFile in self.realDSTest:
      urdu_text = open(textFile, 'r', encoding="utf8").read()
      self.urduListRealTest.append(self.normalizeText(urdu_text))
    self.urduListFakeTestStop=[]
    self.urduListRealTestStop=[]
    for line in self.urduListFakeTest:
      lX=[]
      for word in line:
        if word not in self.stopWords:
          lX.append(word)
      self.urduListFakeTestStop.append(lX)
    for line in self.urduListRealTest:
      lX=[]
      for word in line:
        if word not in self.stopWords:
          lX.append(word)
      self.urduListRealTestStop.append(lX)
  def testNoStopwordRemoval(self):
    '''
    This is the test function where i use the niave bayes formula to predict
    a class for the data and then i check n.o of correct classification and n.o.
    of incorrect classification to calculate accuracy.
    '''
    TotalFakeNews=len(self.urduListFakeTest)
    TotalRealNews=len(self.urduListRealTest)
    fakeClassified=0
    realClassified=0
    for line in self.urduListFakeTest:
      pReal=1
      pFake=1
      for eachWord in line:
        if eachWord in self.fakeBagofWords:
          probFake=self.fakeBagofWords[eachWord]/sum(self.fakeBagofWords.values())
        if eachWord in self.realBagofWords:
          probReal=self.realBagofWords[eachWord]/sum(self.realBagofWords.values())
        pReal*=probReal
        pFake*=probFake
      if pReal*(TotalRealNews/TotalRealNews+TotalFakeNews)>=pFake*(TotalFakeNews/TotalRealNews+TotalFakeNews):
        realClassified+=1
      else:
        fakeClassified+=1
    fakeClassified2=0
    realClassified2=0
    for line in self.urduListRealTest:
      pReal=1
      pFake=1
      for eachWord in line:
        if eachWord in self.fakeBagofWords:
          probFake=self.fakeBagofWords[eachWord]/sum(self.fakeBagofWords.values())
        if eachWord in self.realBagofWords:
          probReal=self.realBagofWords[eachWord]/sum(self.realBagofWords.values())
        pReal*=probReal
        pFake*=probFake
      if pReal*(TotalRealNews/TotalRealNews+TotalFakeNews)>=pFake*(TotalFakeNews/TotalRealNews+TotalFakeNews):
        realClassified2+=1
      else:
        fakeClassified2+=1
    return "Total Accuracy With Stopwords Present: ",((fakeClassified+realClassified2)/(TotalRealNews+TotalFakeNews))*100 
  def testStopwordRemoval(self):
    '''
    This is the test function where i use the niave bayes formula on train data with stop words
    removed to predict a class for the data and then i check n.o of correct classification and n.o.
    of incorrect classification to calculate accuracy.
    '''
    TotalFakeNews=len(self.urduListFakeTest)
    TotalRealNews=len(self.urduListRealTest)
    fakeClassified=0
    realClassified=0
    for line in self.urduListFakeTest:
      pReal=1
      pFake=1
      for eachWord in line:
        if eachWord in self.fakeBagofWordStop:
          probFake=self.fakeBagofWordStop[eachWord]/sum(self.fakeBagofWordStop.values())
        if eachWord in self.realBagofWordStop:
          probReal=self.realBagofWordStop[eachWord]/sum(self.realBagofWordStop.values())
        pReal*=probReal
        pFake*=probFake
      if pReal*(TotalRealNews/TotalRealNews+TotalFakeNews)>=pFake*(TotalFakeNews/TotalRealNews+TotalFakeNews):
        realClassified+=1
      else:
        fakeClassified+=1
    fakeClassified2=0
    realClassified2=0
    for line in self.urduListRealTest:
      pReal=1
      pFake=1
      for eachWord in line:
        if eachWord in self.fakeBagofWordStop:
          probFake=self.fakeBagofWordStop[eachWord]/sum(self.fakeBagofWordStop.values())
        if eachWord in self.realBagofWordStop:
          probReal=self.realBagofWordStop[eachWord]/sum(self.realBagofWordStop.values())
        pReal*=probReal
        pFake*=probFake
      if pReal*(TotalRealNews/TotalRealNews+TotalFakeNews)>=pFake*(TotalFakeNews/TotalRealNews+TotalFakeNews):
        realClassified2+=1
      else:
        fakeClassified2+=1
    return "Total Accuracy after Removing Stop Words: ",((fakeClassified+realClassified2)/(TotalRealNews+TotalFakeNews))*100
  def testNoStopTrainAndTest(self): 
    '''
    This is the test function where i use the niave bayes formula on train data with stop words
    removed from train and as well as test data, to predict a class for the data and then i
    check n.o of correct classification and n.o. of incorrect classification to calculate accuracy.
    '''
    TotalFakeNews=len(self.urduListFakeTest)
    TotalRealNews=len(self.urduListRealTest)
    fakeClassified=0
    realClassified=0
    for line in self.urduListFakeTestStop:
      pReal=1
      pFake=1
      for eachWord in line:
        if eachWord in self.fakeBagofWordStop:
          probFake=self.fakeBagofWordStop[eachWord]/sum(self.fakeBagofWordStop.values())
        if eachWord in self.realBagofWordStop:
          probReal=self.realBagofWordStop[eachWord]/sum(self.realBagofWordStop.values())
        pReal*=probReal
        pFake*=probFake
      if pReal*(TotalRealNews/TotalRealNews)>=pFake*(TotalFakeNews/TotalRealNews+TotalFakeNews):
        realClassified+=1
      else:
        fakeClassified+=1
    fakeClassified2=0
    realClassified2=0
    for line in self.urduListRealTestStop:
      pReal=1
      pFake=1
      for eachWord in line:
        if eachWord in self.fakeBagofWordStop:
          probFake=self.fakeBagofWordStop[eachWord]/sum(self.fakeBagofWordStop.values())
        if eachWord in self.realBagofWordStop:
          probReal=self.realBagofWordStop[eachWord]/sum(self.realBagofWordStop.values())
        pReal*=probReal
        pFake*=probFake
      if pReal*(TotalRealNews/TotalRealNews+TotalFakeNews)>=pFake*(TotalFakeNews/TotalRealNews+TotalFakeNews):
        realClassified2+=1
      else:
        fakeClassified2+=1
    return "Total Accuracy after Removing Stop Words From Train and Test: ",((fakeClassified+realClassified2)/(TotalRealNews+TotalFakeNews))*100
  def test(self,negatiotR):
    '''
    This test function calls all other test functions.
    '''
    self.loadtestData()
    if negatiotR:
      self.removeNegationTest()
    print(self.testNoStopwordRemoval())
    print(self.testStopwordRemoval())
    print(self.testNoStopTrainAndTest())
  def removeNegationTest(self):
    '''
    Removing common urdu negation words in this function from test data
    '''
    urduListFakeTestX=[]
    urduListRealTestX=[]
    urduListFakeTestStopX=[]
    urduListRealTestStopX=[]
    import copy
    urduNegation=['انکار',' نہیں','مت','عدم','فقدان','انکار','نفی','تردید','نہ','سلب','معدوم']
    for line in self.urduListFakeTest:
      lX=[]
      for word in line:
        if word not in urduNegation:
          lX.append(word)
      urduListFakeTestX.append(lX)
    for line in self.urduListRealTest:
      lX=[]
      for word in line:
        if word not in urduNegation:
          lX.append(word)
      urduListRealTestX.append(lX)
    for line in self.urduListFakeTestStop:
      lX=[]
      for word in line:
        if word not in urduNegation:
          lX.append(word)
      urduListFakeTestStopX.append(lX)
    for line in self.urduListRealTestStop:
      lX=[]
      for word in line:
        if word not in urduNegation:
          lX.append(word)
      urduListRealTestStopX.append(lX)
    self.urduListFakeTest=copy.deepcopy(urduListFakeTestX)
    self.urduListRealTest=copy.deepcopy(urduListRealTestX)
    self.urduListFakeTestStop=copy.deepcopy(urduListFakeTestStopX)
    self.urduListRealTestStop=copy.deepcopy(urduListRealTestStopX)
  def makeTrainDataBooleanNB(self):
    '''
    Removing repeating words from train data for Multnomial Naive Bayes Classification. Using Sets.
    '''
    #Training Data Shouldn't have Repetion, remove repitition from individual files
    self.urduListFakeTrain=[]
    for textFile in self.fakeDS:
      urdu_text = open(textFile, 'r', encoding="utf8").read()
      self.urduListFakeTrain.append(self.normalizeText(urdu_text))
    self.urduListRealTrain=[]
    for textFile in self.realDS:
      urdu_text = open(textFile, 'r', encoding="utf8").read()
      self.urduListRealTrain.append(self.normalizeText(urdu_text))
    lXX=[]
    for line in self.urduListFakeTrain:
      lX=set(line)
      line=list(lX)
      lXX.append(line)
    lYY=[]
    for line in self.urduListRealTrain:
      lX=set(line)
      line=list(lX)
      lYY.append(line)
    import copy
    self.urduListFakeTrain=copy.deepcopy(lXX)
    self.urduListRealTrain=copy.deepcopy(lYY)
  def removeNegationTrain(self):
    '''
    Removing negation from train data
    '''
    urduListFakeTrainX=[]
    urduListRealTrainX=[]
    import copy
    urduNegation=['انکار',' نہیں','مت','عدم','فقدان','انکار','نفی','تردید','نہ','سلب','معدوم']
    for line in self.urduListFakeTrain:
      lX=[]
      for word in line:
        if word not in urduNegation:
          lX.append(word)
      urduListFakeTrainX.append(lX)
    for line in self.urduListRealTrain:
      lX=[]
      for word in line:
        if word not in urduNegation:
          lX.append(word)
      urduListRealTrainX.append(lX)
    self.urduListRealTrain=copy.deepcopy(urduListRealTrainX)
    self.urduListFakeTrain=copy.deepcopy(urduListFakeTrainX)
      


Lets call the Multinomial Naive Bayes Classifier without removing Stopwords then removing them from train data and them removing them from test data as well.

In [ ]:
Dir1="/content/drive/My Drive/data/Train/Fake"
Dir2="/content/drive/My Drive/data/Train/Real"
Dir3="/content/drive/My Drive/data/Test/Fake"
Dir4="/content/drive/My Drive/data/Test/Real"
Dir5='/content/drive/My Drive/data/stopwords-ur.txt'
ClassifierX=NaiveBayes(Dir1,Dir2,Dir3,Dir4,Dir5)
ClassifierX.loadTrainData()
ClassifierX.train()
negatiotR=False
ClassifierX.test(negatiotR)

('Total Accuracy With Stopwords Present: ', 57.63358778625955)
('Total Accuracy after Removing Stop Words: ', 58.396946564885496)
('Total Accuracy after Removing Stop Words From Train and Test: ', 59.92366412213741)


Lets call the Boolean Naive Bayes Classifier without removing Stopwords then removing them from train data and them removing them from test data as well.

In [ ]:
ClassifierX.makeTrainDataBooleanNB()
ClassifierX.train()
negatiotR=False
ClassifierX.test(negatiotR)

('Total Accuracy With Stopwords Present: ', 58.396946564885496)
('Total Accuracy after Removing Stop Words: ', 58.396946564885496)
('Total Accuracy after Removing Stop Words From Train and Test: ', 60.30534351145038)


Lets call the Boolean Naive Bayes Classifier while taking care of negation.


In [ ]:
ClassifierX.removeNegationTrain()
ClassifierX.train()
negatiotR=True
ClassifierX.test(negatiotR)

('Total Accuracy With Stopwords Present: ', 58.396946564885496)
('Total Accuracy after Removing Stop Words: ', 58.396946564885496)
('Total Accuracy after Removing Stop Words From Train and Test: ', 60.30534351145038)
